In [6]:
#--------------------------------------------------------------------
# Implements Ball motion prediction using Kalman Filter
#
# Author: Sriram Emarose [sriram.emarose@gmail.com]
#
#
#
#--------------------------------------------------------------------

import cv2 as cv
import numpy as np
import sys

# Instantiate OCV kalman filter
class KalmanFilter:

    kf = cv.KalmanFilter(4, 2)
    kf.measurementMatrix = np.array([[1, 0, 0, 0], [0, 1, 0, 0]], np.float32)
    kf.transitionMatrix = np.array([[1, 0, 1, 0], [0, 1, 0, 1], [0, 0, 1, 0], [0, 0, 0, 1]], np.float32)

    def Estimate(self, coordX, coordY):
        ''' This function estimates the position of the object'''
        measured = np.array([[np.float32(coordX)], [np.float32(coordY)]])
        self.kf.correct(measured)
        predicted = self.kf.predict()
        return predicted



#Performs required image processing to get ball coordinated in the video
class ProcessImage:

    def DetectObject(self):

        vid = cv.VideoCapture(0) 

        if(vid.isOpened() == False):
            print('Cannot open input video')
            return

        width = int(vid.get(3))
        height = int(vid.get(4))

        # Create Kalman Filter Object
        kfObj = KalmanFilter()
        predictedCoords = np.zeros((2, 1), np.float32)

        cont = True

        while(vid.isOpened()):
            rc, frame = vid.read()

            if(rc == True):
                [ballX, ballY] = self.DetectBall(frame)

                predictedCoords = kfObj.Estimate(int(ballX), int(ballY))
                

                predictedCoords = np.resize(np.array([int(predictedCoords[0]),int(predictedCoords[1])]),(1,2))[0]
                print(predictedCoords)

                # np.resize(predictedCoords, (1,2))

                # Draw Actual coords from segmentation
                cv.circle(frame, (int(ballX), int(ballY)), 20, [0,0,255], 2, 8)
                cv.line(frame,(int(ballX), int(ballY + 20)), (int(ballX + 50), int(ballY + 20)), [100,100,255], 2,8)
                cv.putText(frame, "Actual", (int(ballX + 50), int(ballY + 20)), cv.FONT_HERSHEY_SIMPLEX,0.5, [50,200,250])

                # Draw Kalman Filter Predicted output
                cv.circle(frame, (predictedCoords[0], predictedCoords[1]), 20, [0,255,255], 2, 8)
                cv.line(frame, (predictedCoords[0] + 16, predictedCoords[1] - 15), (predictedCoords[0] + 50, predictedCoords[1] - 30), [100, 10, 255], 2, 8)
                cv.putText(frame, "Predicted", (int(predictedCoords[0] + 50), int(predictedCoords[1] - 30)), cv.FONT_HERSHEY_SIMPLEX, 0.5, [50, 200, 250])
                cv.imshow('Input', frame)

                if (cv.waitKey(300) & 0xFF == ord('q')):
                    break

            else:
                break

        vid.release()
        cv.destroyAllWindows()

    # Segment the green ball in a given frame
    def DetectBall(self, frame):

        # Set threshold to filter only green color & Filter it
        lowerBound = np.array([130,30,0], dtype = "uint8")
        upperBound = np.array([255,255,90], dtype = "uint8")
        greenMask = cv.inRange(frame, lowerBound, upperBound)

        # Dilate
        kernel = np.ones((5, 5), np.uint8)
        greenMaskDilated = cv.dilate(greenMask, kernel)
        #cv.imshow('Thresholded', greenMaskDilated)

        # Find ball blob as it is the biggest green object in the frame
        [nLabels, labels, stats, centroids] = cv.connectedComponentsWithStats(greenMaskDilated, 8, cv.CV_32S)

        # First biggest contour is image border always, Remove it
        stats = np.delete(stats, (0), axis = 0)
        try:
            maxBlobIdx_i, maxBlobIdx_j = np.unravel_index(stats.argmax(), stats.shape)

        # This is our ball coords that needs to be tracked
            ballX = stats[maxBlobIdx_i, 0] + (stats[maxBlobIdx_i, 2]/2)
            ballY = stats[maxBlobIdx_i, 1] + (stats[maxBlobIdx_i, 3]/2)
            return [ballX, ballY]
        except:
               pass

        return [0,0]


#Main Function
def main():

    processImg = ProcessImage()
    processImg.DetectObject()


if __name__ == "__main__":
    main()

print('Program Completed!')

[0 0]
[275 137]
[553 268]
[  0 382]
[558 314]
[658 289]
[633 278]
[591 274]
[574 266]
[560 276]
[564 279]
[552 279]
[558 278]
[557 277]
[556 280]
[556 278]
[556 276]
[554 277]
[559 268]
[559 267]
[517 204]
[639 234]
[505 182]
[290 442]
[253 449]
[319 383]
[336 358]
[360 348]
[373 351]
[344 353]
[338 352]
[337 352]
[338 356]
[340 363]
[361 351]
[464 249]
[465 226]
[440 237]
[461 260]
[476 278]
[477 271]
[508 267]
[321 413]
[295 424]
[311 411]
[329 395]
[360 389]
[343 386]
[334 385]
[335 383]
[334 384]
[342 383]
[344 383]
[337 383]
[362 385]
[345 385]
[361 385]
[343 385]
[359 385]
[359 381]
[362 382]
[524 209]
[536 182]
[629 501]
[450 123]
[347  99]
[511 298]
[397 226]
[521 346]
[357 410]
[520 357]
[373 383]
[349 381]
[357 376]
[373 371]
[382 370]
[384 375]
[554 326]
[572 319]
[555 325]
[534 329]
[360 383]
[601 343]
[-79 203]
[556 235]
[711 248]
[608 174]
[594 213]
[642 267]
[609 347]
[511 115]
[386  33]
[138 405]
[ 63 273]
[159 369]
[-41  52]
[-53 206]
[-23 251]
[  2 256]
[ 15 248]
[ 20